<a href="https://colab.research.google.com/github/akshaypt7/customer_relationship_prediction/blob/main/KDD_cup_boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [161]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

from IPython.display import display, HTML

In [162]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [163]:
df_train = pd.read_csv('/content/drive/MyDrive/UnivAI/ML-2 project/data/orange_small_train.data', sep='\t')

In [164]:
df_train.shape

(50000, 230)

In [165]:
df_train.head()

Var1  Var2  Var3  Var4  Var5  ...  Var226   Var227         Var228  Var229  Var230
0   NaN   NaN   NaN   NaN   NaN  ...    xb3V     RAYp  F2FyR07IdsN7I     NaN     NaN
1   NaN   NaN   NaN   NaN   NaN  ...    fKCe     RAYp  F2FyR07IdsN7I     NaN     NaN
2   NaN   NaN   NaN   NaN   NaN  ...    Qu4f  02N6s8f  ib5G6X1eUxUn6    am7c     NaN
3   NaN   NaN   NaN   NaN   NaN  ...    FSa2     RAYp  F2FyR07IdsN7I     NaN     NaN
4   NaN   NaN   NaN   NaN   NaN  ...    FSa2     RAYp  F2FyR07IdsN7I    mj86     NaN

[5 rows x 230 columns]

In [166]:
df_test = pd.read_table('/content/drive/MyDrive/UnivAI/ML-2 project/data/orange_small_test.data')
df_test.head()

Var1  Var2  Var3  Var4  ...  Var227               Var228  Var229  Var230
0   NaN   NaN   NaN   NaN  ...    ZI9m     R4y5gQQWY8OodqDV     NaN     NaN
1   NaN   NaN   NaN   NaN  ...    RAYp        F2FyR07IdsN7I     NaN     NaN
2   NaN   NaN   NaN   NaN  ...    RAYp        F2FyR07IdsN7I     NaN     NaN
3   NaN   NaN   NaN   NaN  ...    RAYp  TCU50_Yjmm6GIBZ0lL_    am7c     NaN
4   NaN   NaN   NaN   NaN  ...    ZI9m           iyHGyLCEkQ    am7c     NaN

[5 rows x 230 columns]

In [167]:
y_train_appetency =pd.read_table('/content/drive/MyDrive/UnivAI/ML-2 project/data/orange_small_train_appetency.labels',header = None).loc[:, 0].astype('category')
y_train_appetency.head()

0   -1
1   -1
2   -1
3   -1
4   -1
Name: 0, dtype: category
Categories (2, int64): [-1, 1]

In [168]:
y_train_upselling= pd.read_table('/content/drive/MyDrive/UnivAI/ML-2 project/data/orange_small_train_upselling.labels', header = None).loc[:, 0].astype('category')
y_train_upselling.head()

0   -1
1   -1
2   -1
3   -1
4   -1
Name: 0, dtype: category
Categories (2, int64): [-1, 1]

In [169]:
y_train_churn = pd.read_table('/content/drive/MyDrive/UnivAI/ML-2 project/data/orange_small_train_churn.labels', header = None).loc[:, 0].astype('category')
y_train_churn.head()

0   -1
1    1
2   -1
3   -1
4   -1
Name: 0, dtype: category
Categories (2, int64): [-1, 1]

### Preprocessing

In [170]:
def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )

In [171]:
def tbl_report(tbl, cols=None, card=10):
    print("Table Shape", tbl.shape)
    dtypes = tbl.dtypes
    nulls = []
    uniques = []
    numuniques = []
    vcs = []
    for col in dtypes.index:
        n = tbl[col].isnull().sum()
        nulls.append(n)
        strdtcol = str(dtypes[col])
        #if strdtcol == 'object' or strdtcol[0:3] == 'int' or strdtcol[0:3] == 'int':
        #print(strdtcol)
        uniqs = tbl[col].unique()
        uniquenums = uniqs.shape[0]
        if uniquenums < card: # low cardinality
            valcounts = pd.value_counts(tbl[col], dropna=False)
            vc = "\n".join(["{}:{}".format(k,v) for k, v in valcounts.items()])
        else:
            vc='HC' # high cardinality
        uniques.append(uniqs)
        numuniques.append(uniquenums)
        vcs.append(vc)
    nullseries = pd.Series(nulls, index=dtypes.index)
    uniqueseries = pd.Series(uniques, index=dtypes.index)
    numuniqueseries = pd.Series(numuniques, index=dtypes.index)
    vcseries = pd.Series(vcs, index=dtypes.index)
    df = pd.concat([dtypes, nullseries, uniqueseries, numuniqueseries, vcseries], axis=1)
    df.columns = ['dtype', 'nulls', 'uniques', 'num_uniques', 'value_counts']
    if cols:
        return df[cols]
    return df


In [172]:
pretty_print(tbl_report(df_train, cols=['dtype', 'nulls', 'num_uniques', 'value_counts']))

Table Shape (50000, 230)


In [173]:
df_table = tbl_report(df_train, cols=['dtype', 'nulls', 'num_uniques', 'value_counts']) # Using the second tbl_report function

df_table['dtype'].value_counts()

Table Shape (50000, 230)


float64    191
object      38
int64        1
Name: dtype, dtype: int64

In [174]:
df_float = df_table[df_table['dtype']== 'float64']
lst_empty_float_cols = list(df_float[df_float['nulls'] > len(df_train)*0.95 ].index)
print("Number of float cols where more than 95% of data is null = ",len(lst_empty_float_cols))

Number of float cols where more than 95% of data is null =  149


In [175]:
df_object = df_table[df_table['dtype']== 'object']
lst_empty_categorical_cols = list(df_object[df_object['nulls'] > len(df_train)*0.95 ].index)
print("Number of categorical cols where more than 95% of data is null = ",len(lst_empty_categorical_cols))

Number of categorical cols where more than 95% of data is null =  4


For now, let us drop all the categorical cols and float cols with >95% of data being null

In [176]:
categorical_cols = list(df_table[df_table['dtype']=='object'].index)
drop_cols = lst_empty_float_cols + categorical_cols
print("Total columns to be dropped =",len(drop_cols))

Total columns to be dropped = 187


In [177]:
X_train = df_train.drop(drop_cols,axis=1)
X_train = df_train_kdd.fillna(df_train_kdd.mean())

In [178]:
X_train.head()

Var6  Var7   Var13        Var21  ...    Var163  Var173  Var181      Var189
0  1526.000000   7.0   184.0   464.000000  ...   38418.0     0.0     0.0  462.000000
1   525.000000   0.0     0.0   168.000000  ...  238572.0     0.0     0.0  270.142137
2  5236.000000   7.0   904.0  1212.000000  ...  434946.0     0.0     0.0  270.142137
3  1326.437116   0.0     0.0   234.518225  ...       0.0     0.0     0.0  270.142137
4  1029.000000   7.0  3216.0    64.000000  ...       0.0     0.0     0.0  270.142137

[5 rows x 43 columns]

Change the lables from [-1, 1] to [0, 1]

In [179]:
y_train_appetency = (y_train_appetency.astype('int') + 1)/2
y_train_churn = (y_train_churn.astype('int') + 1)/2
y_train_upselling = (y_train_upselling.astype('int') + 1)/2

In [180]:
y_train_appetency.tail()

49995    1.0
49996    1.0
49997    0.0
49998    0.0
49999    0.0
Name: 0, dtype: float64

In [181]:
np.mean(y_train_appetency == 1) , np.mean(y_train_churn == 1) , np.mean(y_train_upselling == 1)

(0.0178, 0.07344, 0.07364)

**All customers churn, we send everyone**

In [182]:
def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=5, score_func=None):
    if score_func:
        print("SCORE FUNC", score_func)
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds)
    gs.fit(X, y)
    print("BEST", gs.best_params_, gs.best_score_)
    best = gs.best_estimator_
    return best

In [183]:
def do_classify(clf, parameters, df, 
                featurenames, 
                score_func, n_folds=5, n_jobs=1):
    # x test and ytest
    Xtrain= df
    ytrain= y_train_churn
    # if not mask:
    #     print("using reuse split")
    #     Xtrain, Xtest, ytrain, ytest = reuse_split['Xtrain'][featurenames], reuse_split['Xtest'][featurenames], reuse_split['ytrain'], reuse_split['ytest']
    # else: # mask is to be used
    #     print("using mask")
    #     Xtrain, Xtest, ytrain, ytest = X[reuse_split], X[~reuse_split], y[reuse_split], y[~reuse_split]
    if parameters:
        clf = cv_optimize(clf, parameters, Xtrain, ytrain, n_jobs=n_jobs, n_folds=n_folds, score_func=score_func)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    # test_accuracy = clf.score(Xtest, ytest)
    print("############# based on standard predict 0.5 ################")
    print("Accuracy on training data: %0.2f" % (training_accuracy))
    # print("Accuracy on test data:     %0.2f" % (test_accuracy))
    # print(confusion_matrix(ytest, clf.predict(Xtest)))
    # print("########################################################")
    # return clf, dict(Xtrain=Xtrain, ytrain=ytrain, Xtest=Xtest, ytest=ytest)

In [184]:
pure_cat_vars = []
cont_vars = X_train.columns

In [185]:
# set up standardization
ss = StandardScaler()
# oe hot encoding
oh = OneHotEncoder()
# continuous variables need to be standardized
cont_pipe = Pipeline([("scale", ss)])
# categorical variables need to be one hot encoded
cat_pipe = Pipeline([('onehot', oh)])
# combine both into a transformer
transformers = [('cont', cont_pipe, cont_vars)]
# apply transformer to relevant columns. Nothing will be done for the rest
ct = ColumnTransformer(transformers=transformers, remainder="passthrough")
# create a pipeline so that we are not leaking data from validation to train in the individual folds
pipe = Pipeline(steps=[('ct', ct), ('model', LogisticRegression(max_iter=10000, penalty='l2'))])
# in paramgrid we dont use C but use model__C corresponding to the name in the pipeline
paramgrid = dict(model__C=[1000, 100, 10, 1, 0.1, 0.01, 0.001])

In [186]:
from sklearn.metrics import roc_auc_score
do_classify(pipe, paramgrid, X_train, 
                              cont_vars, 
                              score_func='roc_auc')

SCORE FUNC roc_auc
BEST {'model__C': 1} 0.6455963945396483
############# based on standard predict 0.5 ################
Accuracy on training data: 0.93


In [187]:
# do_classify(clf, parameters, indf, 
#                 featurenames, targetname, target1val, 
#                 reuse_split, mask=False, 
#                 score_func=None, n_folds=5, n_jobs=1)

In [188]:
# Table report after dropping columns and filling na with 0
pretty_print(tbl_report(df_train_kdd, cols=['dtype', 'nulls', 'num_uniques', 'value_counts'])) 

Table Shape (50000, 43)


# Gradient Boosting

In [189]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gb = GradientBoostingClassifier(n_estimators=1000, max_depth=5)
gb.fit(X_train, y_train_churn)